# Evolution du nombre de prénom masculin et féminin donnés par département au cours du XX siècle

# Libraires

In [1]:
# !pip install altair
# !pip install vega_datasets

import pandas as pd
import numpy as np

import altair as alt

# Table des noms associés à chaque numéros de département

In [2]:
data= pd.read_csv("departements-france.csv")
data.rename(columns = {'code_departement':'dpt'}, inplace = True)
data = data[['dpt', 'nom_departement']]
data

dpt          nom_departement
0     01                      Ain
1     02                    Aisne
2     03                   Allier
3     04  Alpes-de-Haute-Provence
4     05             Hautes-Alpes
..   ...                      ...
96   971               Guadeloupe
97   972               Martinique
98   973                   Guyane
99   974               La Réunion
100  976                  Mayotte

[101 rows x 2 columns]

# Data management

In [3]:
# import de notre jeux de données
df = pd.read_csv("dpt2020.csv", sep=';')

# group by par années puis par département puis par sexe
df_3 = df.groupby(by=['annais', 'dpt', 'sexe']).sum()

# réinitialisaton de l'index
df_3 = df_3.reset_index()

# suppréssion des format d'années non conforme
df_3 = df_3[df_3['annais'] != 'XXXX']

# changer le format de la colonne annais
df_3['annais'] = df_3['annais'].astype(np.int64)

# récupérer les différentes date
annees = df_3.annais.unique().tolist()

# travailler uniquement sur des intervals de dizaine d'années
annees_final = [x for x in annees if x % 10 == 0]
df_3bis = df_3.query('annais == @annees_final')

# # changement de nom de feature
df_3bis = df_3bis.copy()
df_3bis.rename(columns = {'sexe':'sex', 'nombre':'people'}, inplace = True)

# jointure avec la table des noms des département
df_3 = pd.merge(df_3bis, data, on='dpt', how='left') 
df_3 

annais  dpt  sex  people nom_departement
0       1900   01    1    1336             Ain
1       1900   01    2    1837             Ain
2       1900   02    1    2161           Aisne
3       1900   02    2    2911           Aisne
4       1900   03    1    1684          Allier
...      ...  ...  ...     ...             ...
2499    2020  972    2     903      Martinique
2500    2020  973    1    2938          Guyane
2501    2020  973    2    2553          Guyane
2502    2020  974    1    5265      La Réunion
2503    2020  974    2    4947      La Réunion

[2504 rows x 5 columns]

# Liste des 15 premiers département ayant la moyenne de population la plus base

In [4]:
top_mean = df_3[['nom_departement','people']].groupby(['nom_departement']).mean().sort_values(by=['people'], ascending=False).tail(15).index.tolist()
df_3 = df_3.query('nom_departement in @top_mean').sort_values(by=['people'], ascending=False)
df_3

annais  dpt  sex  people nom_departement
2500    2020  973    1    2938          Guyane
2501    2020  973    2    2553          Guyane
1078    1950   70    1    2287     Haute-Saône
1048    1950   55    1    2244           Meuse
1079    1950   70    2    2198     Haute-Saône
...      ...  ...  ...     ...             ...
2351    2020   23    2      76          Creuse
2350    2020   23    1      73          Creuse
2203    2010   48    2      67          Lozère
2400    2020   48    1      52          Lozère
2401    2020   48    2      42          Lozère

[390 rows x 5 columns]

# pyramide

In [5]:
slider = alt.binding_range(min=1900, max=2000, step=10)
select_year = alt.selection_single(name='annais', fields=['annais'],bind=slider, init={'annais': 2000})

base = alt.Chart(df_3).add_selection(
    select_year
).transform_filter(
    select_year
).transform_calculate(
    gender=alt.expr.if_(alt.datum.sex == 1, 'Male', 'Female')
).properties(
    width=250
)

color_scale = alt.Scale(domain=['Male', 'Female'],
                        range=['#1f77b4', '#e377c2'])

left = base.transform_filter(
    alt.datum.gender == 'Female'
).encode(
    y=alt.Y('nom_departement:O', axis=None),
    x=alt.X('sum(people):Q',
            title='nombre de prénom féminin',
            sort=alt.SortOrder('descending')),
    color=alt.Color('gender:N', scale=color_scale, legend=None)
).mark_bar().properties(title='Féminin')

middle = base.encode(
    y=alt.Y('nom_departement:O', axis=None),
    text=alt.Text('nom_departement:O'),
).mark_text().properties(width=150)

right = base.transform_filter(
    alt.datum.gender == 'Male'
).encode(
    y=alt.Y('nom_departement:O', axis=None),
    x=alt.X('sum(people):Q', 
            title='nombre de prénom masculin'),
    color=alt.Color('gender:N', scale=color_scale, legend=None)
).mark_bar().properties(title='Masculin')

alt.concat(left, middle, right, spacing=5, title='Evolution du nombre de prénom masculin et féminin par département au cours des 20ieme siècle')

alt.ConcatChart(...)

### Y a-t-il des effets de genre dans les données ? La popularité des noms donnés aux deux sexes évolue-t-elle de manière cohérente ?

Chaque département possède la même proportion de prénom masculin et féminin données au cours du temps. Le nombre de prénom masculin et féminin données aux nouveaux nés augmente au cours du temps dans tous les départements : cela est dû à l'augmentation de la population global dans le pays. Cependant nous pouvons remarquer un fort rebont de la population en Guyane, nous sommes passé d'une population de 150 nouveaux nés avec un prénom masculin et 150 nouveaux nés avec un prénom féminin en 1900 à plus de 2000 nouveaux nés avec un prénom masculin et près de 1800 nouveaux nés avec un prénom féminin.